In [73]:
import pandas as pd
from bs4 import BeautifulSoup
import re

In [ ]:

def clean_html_tags(data: str) -> str:
    """Remove HTML tags from textual data"""
    if not isinstance(data, str):
        return data  
        
    if not data:
        return
    
    soup = BeautifulSoup(data, "html.parser")
    return soup.get_text()

def remove_hashtags(text: str) -> str:
    """Remove hashtags from textual data"""
    if not isinstance(text, str):
        return text
        
    if not text:
        return
    
    return re.sub(r"#\w+", "", text)

def count_separator_rows(patterns: list, column: pd.Series) -> int:
    """
    Count rows in column with with specified separators. 
    Prints the total number of rows in the column, the number of rows with each separator and the total number of rows with any of the separators.
    """
    
    column_len = len(column)
    print(f"Number of rows in column: {column_len}")

    rows_with_pattern_separators = {}
    for pattern in patterns:
        rows_with_pattern_separator = column.str.contains(pattern, regex=True, na=False)
        rows_with_pattern_separators[pattern] = rows_with_pattern_separator
        print(f"Number of rows with {repr(pattern)} separator: {rows_with_pattern_separator.sum()}")
        
    # series of false with len of step_column. we use OR on it and count the number of rows with any of the separators listed
    rows_with_either = pd.Series([False] * column_len)

    for pattern, pattern_matches in rows_with_pattern_separators.items():
        rows_with_either = rows_with_either | pattern_matches
        
    count_either = rows_with_either.sum()
    print(f"Number of rows with either ',.' or newlines: {count_either}")
    
    return count_either

def split_steps(steps: str, separators: list, ends_with_dot: bool = True) -> list:
    """Split textual data into a list based on provided separators."""
    if not isinstance(steps, str):
        return steps
    
    print("Steps: ", steps)
    # escape to match the literal characters
    pattern = "|".join(separators)

    print("Pattern: ", pattern)
    
    steps_list = re.split(pattern, steps)
    print("Steps list: ", steps_list)
    
    # ensure consistency, all steps end with a dot
    steps_list = [step.strip() for step in steps_list if step.strip()]
    
    # \.,|\n\n
    if ends_with_dot:
        for i in range(len(steps_list)):
            if not steps_list[i].endswith('.'):
                steps_list[i] += '.'
    
    return steps_list
    

In [75]:
df = pd.read_csv("../data/Recipes.csv")
df.sample(5)

,id,name,name.1,author_note,ingredients,steps
6,17,Segedínský guláš,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Česnek, Cibule, Hladká mouka, Houskové knedlík...",Maso nakrájíme na kostky o hraně 2 cm. Slaninu...
41,53,Telecí na paprice,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Bobkový list, Brambory typ A, B, C, Černý pepř...",Cibuli nakrájíme najemno. Maso nakrájíme na ko...
60,72,Telecí na žampionech,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Bobkový list, Brambory typ A, B, C, Cibule, Hl...",Maso nakrájíme na kostky o hraně 4 cm a dáme d...
58,70,Roastbeef,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Bageta, Brambory typ A, B, C, Černý pepř celý,...","Hotový rostbíf vyndáme z trouby, odstraníme pr..."
84,98,Alsaská pizza,Roman Vaněk,S Rohlíkem jsme dali dohromady výběr nejoblíbe...,"Čerstvě drcený pepř, Čerstvé droždí, Červená c...","Gril rozpálíme na maximum a přiklopíme ho., Ka..."


In [76]:
df = df.rename(columns={"name.1": "author_name"}) # rename column as there were duplicate names

In the provided dataset, the "author_note" column contains promotional, repetitive content. While future iterations of the dataset might include recipe-related information, this is not the case for now. Therefore, dropping this column is justified. 

This action improves the efficiency of future usage of the dataset for RAG, by reducing context size for LLM and reducing "noise" in embedding vectors for Vector Search.

In [77]:
df.drop(columns=["author_note"], inplace=True)

During manual inspection, I found that "author_note" column had hashtags and html tags in it. Let's ensure that all columns won't have them.

In [78]:
for column in df.columns:        
    df[column] = df[column].apply(clean_html_tags)
    df[column] = df[column].apply(remove_hashtags)
    
df.sample(5)

,id,name,author_name,ingredients,steps
20,31,Gulášová polévka,Roman Vaněk,"Brambory typ A, B, C, Česnek, Cibule, Drcený k...",Brambory nakrájíme na kostky o hraně 1 cm a sp...
96,110,Souvlaki,Roman Vaněk,"Čerstvě drcený pepř, Čerstvé oregano, Čerstvý ...","Hotové špízy položíme na talíř, zakápneme zbyl..."
46,58,Zatloukaný řízek a petrželové brambory,Roman Vaněk,"Brambory typ A, B, C, Čerstvě drcený pepř, Cit...",Brambory nakrájíme na stejné kousky a uvaříme ...
45,57,Francouzská cibulačka,Roman Vaněk,"Bageta, Bobkový list, Čerstvě drcený pepř, Cib...",Bagetu nakrájíme na plátky 6 mm silné a rozlož...
28,39,Kuře s rajčaty a olivami,Roman Vaněk,"Celé kuře, Černé olivy bez pecky, Čerstvě drce...",Do kastrolu vyskládáme zpět opečené maso kůží ...


During the manual inspection of the dataset, it was found that the steps of cooking are separated either with ",." or new lines. Let us find out whether it is applicable to all recipes: 

In [79]:
count_separator_rows([".,", "\n\n"], df["steps"])

Number of rows in column: 100
Number of rows with '.,' separator: 100
Number of rows with '\n\n' separator: 41
Number of rows with either ',.' or newlines: 100


np.int64(100)

Let's separate: 

In [ ]:
df["steps"] = df["steps"].apply(split_steps, args=([r"\.,", r"\n\n"],))

Pattern:  \.,|\n\n
Steps list:  ['Cibuli nakrájíme nahrubo. Maso nakrájíme na kostky o hraně 4 cm', ' Jakmile je maso měkké, promneme v dlaních do guláše majoránku, dle potřeby znovu osolíme a opepříme, promícháme a ještě 5 minut vaříme. Hotový guláš necháme odpočinout alespoň 15 minut', ' Poté kastrol vrátíme na vyšší oheň, přidáme maso a za častého míchání restujeme, dokud se ze dna kastrolu neodpaří veškerá šťáva, kterou maso pustí. Jakmile zbyde na dně kastrolu opět jen tuk, stáhneme oheň na střední stupeň a restujeme maso dalších 5 minut za častého míchání dozlatova', ' Přilijeme tolik vody, aby její hladina dosahovala 1 cm nad orestované maso. Vše důkladně promícháme plochou vařečkou ze dna kastrolu, aby se vytvořený nápek rozpustil', ' Připravenou hmotu položíme na potravinářskou fólii a pevně zabalíme do tvaru bonbonu. Ten ještě zabalíme do jedné vrstvy fólie', ' Přivedeme k varu, přikryjeme pokličkou, stáhneme oheň na minimum a dusíme na velmi mírn']
Pattern:  \.,|\n\n
Steps l

However, there arises the problem. The plan is to create retrieval mechanism that will allow the chatbot not only to "retell" the recipe, but also retrieve the full recipe from the database and show it to the user in a formatted way, to be as precise as possible and minimise hallucinations. In this case, promotional messages would be benefitial. 

As we currently have no metadata for the dataset, there is no way to determine the purpose of the "id" field. 

In [ ]:
df.to_csv("../data/processed/Recipes_processed.csv", index=False)